In [4]:
import numpy as np
def RFF(X, D, sameFunction):
    np.random.seed(0)
    n_features = len(X)
    
    if sameFunction:
        omegas1 = (np.random.normal(size=(1, D)))
        omegas = omegas1
        
        for i in range(n_features-1):
            omegas = np.vstack([omegas, omegas1])
            
    else:
        omegas = (np.random.normal(size=(n_features, D)))
        
    b = np.random.uniform(0, 2 * np.pi, size=(D,1))
    X_new = np.transpose(omegas)@X+b
    X_new = np.cos(X_new)
    X_new *= np.sqrt(2) / np.sqrt(D)
    
    return X_new

In [5]:
import pandas as pd

df = pd.read_csv("../data/non_linear_ts_lags.csv")
df_short = df.iloc[:14]

In [26]:
A = RFF(df_short.T, 2, False).T.to_numpy()

In [61]:
Q, R = np.linalg.qr(A, mode="complete")

In [85]:
A_0 = A[:1,:].reshape(2, 1)
pinv_A_0 = np.linalg.pinv(A[:1,:])

In [63]:
q.shape, diag

((14, 1), 13)

In [64]:
q[13], q[12]

(array([-3.46944695e-18]), array([1.38777878e-17]))

In [75]:
from copy import deepcopy
G = np.identity(A.shape[0])
G_all = deepcopy(G)
diag = R.shape[0]-1
q = Q[0,:].reshape(Q.shape[1],1)

for i in range(diag,0,-1):
    x = q[i-1, 0]
    y = q[i, 0]
    r = np.sqrt(x**2 + y**2)
    c = x / r
    s = -y / r

    G[i-1, i-1] = c
    G[i-1, i]   = -s
    G[i, i-1]   = s
    G[i, i]     = c

    G_all = G_all @ G.T
    q = G @ q
    G = np.identity(Q.shape[0])

#all_Q = all_Q @ G_all

In [77]:
e_1 = np.eye(A.shape[0],1)
G_all.T @ (R - G_all @ e_1 @ A_0)

array([[ 4.47089899e-17, -3.52115400e-16],
       [-2.58610175e+00, -3.79421768e-01],
       [ 1.42749845e-18, -2.59886785e+00],
       [ 1.17518645e-18,  2.50488580e-17],
       [ 9.00916479e-19, -1.90663620e-18],
       [ 4.03644951e-18,  1.76064778e-17],
       [-9.08523681e-18, -6.95062911e-17],
       [ 6.91794478e-18,  3.89760079e-17],
       [-4.08404566e-18, -2.65275487e-17],
       [ 2.50995496e-18,  1.64698592e-17],
       [ 4.99394496e-18,  2.17711904e-17],
       [-3.48744063e-18, -2.36569257e-17],
       [ 5.39808362e-18,  3.12502771e-17],
       [-6.10730008e-19, -1.62455974e-18]])

In [102]:
P = np.linalg.pinv(R)

In [206]:
A_P = A @ P
P_A = P @ A

u = (np.eye(A_P.shape[0], A_P.shape[1]) - A_P) @ e_1
v = -(np.eye(P_A.shape[0], P_A.shape[1]) - P_A) @ A_0
k = P @ e_1
h = -A_0.T @ P
s = -A_0.T @ P @ e_1


In [207]:
p2 = -(u.T@u).item()/s * P @ h.T - k
q2 = -(h@h.T).item()/s * u.T - h
sigma_2 = (h@h.T).item() * (u.T@u).item() + s * s

P_temp = P + 1/s * P @ h.T @ u.T - s/sigma_2 * p2 @ q2

In [251]:
h = P @ G_all @ e_1
k = -A_0.T @ P
s = 1 + -A_0.T @ P @ G_all @ e_1

P_temp = P - 1/s * h * k

In [261]:
A_0

array([[-0.05486408],
       [-0.35336174]])

In [260]:
je = P_temp @ G_all
je @ (G_all.T @ R - e_1 @ A_0.T)

array([[ 1.00000000e+00,  1.87429767e-18],
       [-8.09969504e-17,  1.00000000e+00]])

In [257]:
temp =  (G_all.T @ R - e_1 @ A_0.T)
temp = np.linalg.pinv(temp)
#temp[temp < 1e-16] = 0
temp[0]

array([ 1.25352403e-17, -3.86682388e-01,  5.64537035e-02,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00])

In [243]:
temp = G_all.T @ (R - G_all @ e_1 @ A_0.T)
temp = np.linalg.pinv(temp)
#temp[temp < 1e-16] = 0
temp 

array([[ 1.25040785e-17, -3.86682388e-01,  5.64537035e-02,
        -3.64658794e-19,  1.78995972e-19,  2.33952040e-19,
         1.22435817e-19,  2.09788075e-19, -4.74331971e-20,
         2.55307239e-20,  2.89703805e-19, -1.86823533e-20,
         1.45511219e-19, -5.79753849e-20],
       [-8.52265790e-17,  1.11265561e-17, -3.84782935e-01,
         3.68315360e-18, -3.01862652e-19,  2.51909679e-18,
        -1.00936026e-17,  5.62043164e-18, -3.83889798e-18,
         2.38397058e-18,  3.11491631e-18, -3.42683925e-18,
         4.50959117e-18, -2.27262380e-19]])

In [250]:
che = je @  G_all.T @ (R - G_all @ e_1 @ A_0.T)
che

array([[ 1.00000000e+00,  1.66045083e-17],
       [-8.10118544e-17,  1.00000000e+00]])

In [124]:
R_inv = np.linalg.pinv(R)
R_inv[R_inv < 1e-16] = 0
R_inv

array([[0.3865954 , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.38143217, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ]])

In [ ]:
a = R_inv.T @ A_0.T

In [ ]:
s = 1 + a.T @ a
new_R_inv = R_inv - (1/s) * R_inv @ a @ a.T

In [ ]:
Q, R = np.linalg.qr(A[1:,:], mode="reduced")
ts = np.linalg.pinv(np.linalg.pinv(R))
ts

array([[-2.58610175, -0.37942177],
       [ 0.        ,  2.59886785]])

In [ ]:
ts


array([[-2.58610175, -0.37942177],
       [ 0.        ,  2.59886785]])

In [ ]:
new_R_inv

array([[ 0.38658102, -0.05713136],
       [-0.0010465 ,  0.3749364 ]])

In [ ]:
new_R_inv

array([[ 0.38658102, -0.05713136,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ],
       [-0.0010465 ,  0.3749364 ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ]])

In [ ]:
import scipy

P = scipy.linalg.pinv(A)
#proj_M = P @ A

In [ ]:
A[:1,:].shape

(1, 200)

In [ ]:
A[:1,:].shape

(1, 200)

In [ ]:
tempi = pinv_A_0 @ A[:1,:]
tempi.shape

(200, 200)

In [ ]:
(P @ A) @ pinv_A_0

array([[ 1.03824809e-01],
       [-1.02605530e-01],
       [ 5.41644990e-02],
       [ 1.00372494e-01],
       [ 5.05606869e-02],
       [ 2.64504153e-02],
       [ 8.58378696e-02],
       [-1.03737839e-01],
       [ 3.43021183e-02],
       [ 5.89081731e-02],
       [-1.01926070e-01],
       [-4.15050210e-03],
       [-8.98820884e-03],
       [ 2.96748413e-03],
       [ 7.07864403e-02],
       [-3.62700790e-02],
       [-8.65409095e-02],
       [ 1.01511644e-01],
       [ 6.08858799e-02],
       [-9.14101919e-02],
       [-9.36788493e-02],
       [-7.23195486e-02],
       [ 7.96486533e-02],
       [-1.04203268e-01],
       [ 1.53796743e-03],
       [-4.45500011e-02],
       [ 6.83513174e-02],
       [-3.94167952e-02],
       [-3.89371851e-02],
       [-8.61178318e-02],
       [-4.81173890e-02],
       [ 7.91015343e-02],
       [-4.66302847e-02],
       [-9.50259860e-02],
       [ 2.68525309e-03],
       [ 1.01875838e-01],
       [-1.03896759e-01],
       [-1.04336307e-01],
       [-9.7

In [ ]:
e_1 = np.eye(30,1)


In [ ]:
P.shape

(200, 30)

In [ ]:
A.shape, e_1.shape

((30, 200), (30, 1))

In [ ]:
e_1.T @ A @ A_0.T

array([[0.9584146]])

In [ ]:
A_0 @ P @ e_1

array([[1.]])